# Hello world
## Optimization algorithms: demo

<em><b>Note</b>: This is just a demo of optimization techniques - not related to any real-life problem.</em>

Imagine the world of strings where the environment demands that the strings are very fit. An unfit string cannot survive. 

How do we define the fitness of a string? The environment demands that the string should be as similar as possible to the target string $s$=''Hello, world''. The closer the string is to ''Hello, world'', the better its chances of survival.

<b> The problem:</b> given a set of random strings of length len($s$) over alphabet $\sigma=\{32 \ldots 122\}$, produce the string which best matches the environment, i.e. with a minimum fitness score. 

Let <em>Weighted Hamming Distance</em> between two strings $x$ and $y$ (both of length $n$) be defined as:<br>
$WH(x,y) = \sum_{i=0}^{n} abs(x[i] - y[i])$

$WH(x,y)$ estimates how far is $x$ from $y$. The lower $WH(x,y)$, the closer string $x$ approaches the target string $y$.<br>
In the space of $\sigma^n$ of all possible strings, we are looking for a global minimum - the string $m$ such that $WH(m,s)$ is minimized. <br>

If we compute $WH(x,s)$ for an arbitrary string $x$, we say that we evaluate the <em>fitness</em> of $x$ with respect to the target string $s$. This is our <em>fitness function</em>. 

## 1. Preparation

In [1]:
# flag which can be turned on/off to print the steps of each optimization
print_steps = True

### 1.1. String fitness function 

In [2]:
s = 'Hello, world'
n = len(s)

# fitness function - weighted hamming distance
string_fitness = lambda x: sum([abs(ord(x[i])- ord(s[i])) for i in range(n)])

### 1.2. Generating initial random strings

In [3]:
import random

alphabet = range(32,123)

def get_random_string(n, sigma):    
    t = [chr(random.choice(sigma)) for i in range(n)]
    return ''.join(t)

# build initial population of many random strings of target length n
def get_rand_population(population_size, n, sigma):
    population=[]
    for i in range(0,population_size):
        individual_str = get_random_string(n, sigma)
        population.append(individual_str)
    return population

# test
print(get_random_string(len(s), alphabet))
print(get_rand_population(3, len(s), alphabet ))

_m@TV0__6'=B
['/yxd\\az@3ozg', 'edft8KoQ@Giw', 'S9G%eVUAA?+Z']


## 2. Random optimize

The simplest possible idea is to try a vaste amount of random solutions and select the one with the best fitness score. Let's see how close we can get to the target $s$=''Hello, world'' with this approach.

After all, isn't evolution just a lottery? ''Physics makes rules, evolution rolls the dice'' ( Charles Cockell. "The Equations of Life").

In [4]:
def random_optimize(population, fitness_function):
    best_score = None
    best_solution = None
    for i in range(len(population)):
        # Calculate fitness 
        score = fitness_function(population[i])

        # Compare it to the best one so far
        if best_score == None or score < best_score:
            best_score = score
            best_solution = population[i]
            if print_steps:
                print(best_solution, " -> score:", best_score)
            
    return (best_score, best_solution, len(population))


# Now the test
string_population = get_rand_population(204800, len(s), alphabet)

best_score, best_sol, iterations = random_optimize (string_population, string_fitness)

=!mJE9eY6% 2  -> score: 528
Z)Tx[fvG]U(/  -> score: 494
9iX_>AOxz^30  -> score: 310
oToQv"[UDo6g  -> score: 299
PSbh;r8^@pkA  -> score: 296
,]]jd/Qt6QP\  -> score: 245
Eyevo0\FGrWX  -> score: 226
Igs]e%%+tqlN  -> score: 151
Yi_ho)Bkg^ZW  -> score: 146
AfSju.0m]Xq_  -> score: 123
Mabnd7#n`sFa  -> score: 112
OYiflF-wswtt  -> score: 103


In [5]:
print()
print("*************Rand optimize**************")
print("trials:{}, best solution:'{}', best score:{}".format(iterations,best_sol,best_score))


*************Rand optimize**************
trials:204800, best solution:'OYiflF-wswtt', best score:103


Randomly trying different solutions is very inefficient because the probability of getting anything close to perfect Hello, world is $(\frac{1}{90})^{12}$. No matter how many random guesses you try – the fitness of the resulting string is very low (the distance from the target remains high).

## 3. Hill climbing

We did not advance far with the random optimization. This happens because each time we come up with some solution and evaluate its fitness - we discard it and try another one - not related to the previous.

The idea behing the <em>hill climbing</em> optimization is to start at a random point in space (choose one random string), and try to move from this point into a direction of better fitness. We will generate a set of neighbors for a randomly selected string, and see if any of these neighbors improve the overall fitness score of the string. 

We continue moving into the direction of this better solution, until there is no more improvement. 

<img src="images/hillclimbing.jpg" style="height:200px;">


The algorithm may produce an optimal solution, but it may also get stuck at the local minimum.
<img src="images/localmin.jpg" style="height:200px;">

In [6]:
def hillclimb_optimize(start_point, fitness_function):
    sol = start_point
    score = fitness_function(sol)
    iterations = 0
    
    # Main loop
    while 1:
        iterations += 1
        
        current_best_score = score
        # Create  neighboring solutions
        neighbors=[]

        for j in range(len(s)):
            # Change character at position j one up or one down
            if ord(sol[j])>32:
                neighbors.append(sol[0:j]+chr(ord(sol[j])-1)+sol[j+1:])
            if ord(sol[j])<122:
                neighbors.append(sol[0:j]+chr(ord(sol[j])+1)+sol[j+1:])
        
        for n_str in neighbors:
            n_score = fitness_function(n_str)
            if n_score < score:
                score = n_score
                sol = n_str
                if print_steps:
                    print(sol, " -> score:", score)      
        
        # If there's no improvement, then we've reached the bottom of the hill
        if score == current_best_score:
            break        
        
        if score == 0:
            break        
        
    return (score, sol, iterations)

# Now the test

rand_str = get_random_string(len(s), alphabet)
best_score, best_sol, iterations = hillclimb_optimize(rand_str, string_fitness)

q3V4vH$Jt=;P  -> score: 380
p3V4vH$Jt=;P  -> score: 379
o3V4vH$Jt=;P  -> score: 378
n3V4vH$Jt=;P  -> score: 377
m3V4vH$Jt=;P  -> score: 376
l3V4vH$Jt=;P  -> score: 375
k3V4vH$Jt=;P  -> score: 374
j3V4vH$Jt=;P  -> score: 373
i3V4vH$Jt=;P  -> score: 372
h3V4vH$Jt=;P  -> score: 371
g3V4vH$Jt=;P  -> score: 370
f3V4vH$Jt=;P  -> score: 369
e3V4vH$Jt=;P  -> score: 368
d3V4vH$Jt=;P  -> score: 367
c3V4vH$Jt=;P  -> score: 366
b3V4vH$Jt=;P  -> score: 365
a3V4vH$Jt=;P  -> score: 364
`3V4vH$Jt=;P  -> score: 363
_3V4vH$Jt=;P  -> score: 362
^3V4vH$Jt=;P  -> score: 361
]3V4vH$Jt=;P  -> score: 360
\3V4vH$Jt=;P  -> score: 359
[3V4vH$Jt=;P  -> score: 358
Z3V4vH$Jt=;P  -> score: 357
Y3V4vH$Jt=;P  -> score: 356
X3V4vH$Jt=;P  -> score: 355
W3V4vH$Jt=;P  -> score: 354
V3V4vH$Jt=;P  -> score: 353
U3V4vH$Jt=;P  -> score: 352
T3V4vH$Jt=;P  -> score: 351
S3V4vH$Jt=;P  -> score: 350
R3V4vH$Jt=;P  -> score: 349
Q3V4vH$Jt=;P  -> score: 348
P3V4vH$Jt=;P  -> score: 347
O3V4vH$Jt=;P  -> score: 346
N3V4vH$Jt=;P  -> sco

In [7]:
print()
print("*************Hill climbing****************")
print("steps:{}, best solution:'{}', best score:{}".format(iterations,best_sol,best_score))


*************Hill climbing****************
steps:381, best solution:'Hello, world', best score:0


## 3. Simulated annealing
The idea of the <em>simulated annealing</em> is borrowed from physics. In metalurgical annealing we heat a metal (alloy) to a very high temperature, the crystal bonds break, and the atoms diffuse more freely. If we cool it slowly, the atoms tend to form more regular crystals producing an alloy with the low termodynamics energy.  

<img style="height:250px;float:right;padding:4px;" src="images/sim_annealing.jpg" >

In the <em>simulated annealing</em> algorithm we set the initial temperature very high, and then we generate a single random neighbor of the current solution. The fitness of this neihgbor can be better or worse that that of the current solution. When the temperature is high, the probability of selecting worse solution is higher. This allows to better explore the search space and get out of the local minimum. The temperature gradually decreases, and so at the end we do not accept worse solutions.

The criterion of accepting ''bad'' solutions:

$p=e^{\frac{-\Delta F}{T}}>R(0,1)$

where $T$ is the current temperature, $R(0,1)$ is a random number between $0$ and $1$, and $\Delta F$ is the difference between the fitness score of new solution and the old solution.

Since the temperature $T$ (the willingness to accept a worse solution) starts very high,
the exponent will be close to 0, and $p$ will almost be 1. As the temperature decreases, the difference between the new fitness score and the old one becomes more important - a bigger difference leads to a lower probability, so the
algorithm will not accept solutions which do not improve fitness - converging to a local minimum after exploring a large global search space.

In [12]:
import math

def annealing_optimize(start_sol, fitness_function, T=10000.0, cool=0.95, step=1):    
    sol = start_sol
    iterations = 0
    
    # Graduate cooling
    while T > 0.01: 
        score = fitness_function(sol)
        
        # Choose one of spots randomly
        i = random.randint(0, len(sol) - 1)

        # Choose rand direction to change the character at position i
        dir = random.randint(-step, step)

        change = ord(sol[i]) + dir
        # out of domain
        if change > 122 or change < 32:
            continue
    
        iterations += 1
        
        # Create a new solution with one of the characters changed
        new_sol = sol[:i] + chr(change) + sol[i+1:]

        # Calculate the new cost
        new_score = fitness_function(new_sol)        

        # Does it make the probability cutoff?
        p = pow(math.e, -(new_score - score) / T)
        
        if new_score < score or p > random.random() :
            sol = new_sol
            score = new_score
            if print_steps:
                print(sol, "-> score:", score)       
        
        if score == 0:
            break
            
        # Decrease the temperature
        T = T * cool
    
    return (score, sol, iterations)

# now test
rand_str = get_random_string(len(s), alphabet)
best_score, best_sol, iterations = annealing_optimize(rand_str, string_fitness,
                                          T=204800.0, cool=0.999, step=1)

rKSj:3Lvibv@ -> score: 268
rKSj:3Lvibv@ -> score: 268
rKSj:3Lvjbv@ -> score: 267
rKSj:3Lvjcv@ -> score: 266
rKSj:3Lvjcv@ -> score: 266
rKSj:3Lvjcv@ -> score: 266
rKSi:3Lvjcv@ -> score: 267
rKSi:3Lvjcv@ -> score: 267
rKSi:3Lvjcv@ -> score: 267
rKSi:3Lvjcw@ -> score: 268
rKSi:3Mvjcw@ -> score: 269
rKSi:3Mvicw@ -> score: 270
rKSi:3Mvicw@ -> score: 270
rKSi:3Mvicw@ -> score: 270
rKSi:3Mvicw@ -> score: 270
rKTi:3Mvicw@ -> score: 269
rKTi:3Mvicw@ -> score: 269
sKTi:3Mvicw@ -> score: 270
sKTi:3Muicw@ -> score: 271
sKTi:3Mujcw@ -> score: 270
sKTh:3Mujcw@ -> score: 271
sKTh:2Mujcw@ -> score: 270
sKTh:2Mujcx@ -> score: 271
sKTh:2Lujcx@ -> score: 270
sKTh:2Lujcx@ -> score: 270
sKTh:2Lujdx@ -> score: 269
sKTh:2Lujex@ -> score: 268
sKTh;2Lujex@ -> score: 267
sKTh;2Lujex@ -> score: 267
sKTi;2Lujex@ -> score: 266
sKTi;2Lukex@ -> score: 265
sKTi;2Lukdx@ -> score: 266
sKTi;2Lvkdx@ -> score: 265
sKTi;2Lvkdx? -> score: 266
sKSi;2Lvkdx? -> score: 267
rKSi;2Lvkdx? -> score: 266
rKSi;2Kvkdx? -> score: 265
s

nPSm;*EuccyI -> score: 245
nPSm;*EuccyI -> score: 245
nPSm;*EuccyI -> score: 245
nPSm;*EubcyI -> score: 246
nPSm;)EubcyI -> score: 247
nPSm;)EubcyI -> score: 247
mPSm;)EubcyI -> score: 246
mPSm<)EubcyI -> score: 245
mPSm<)EubcyI -> score: 245
mPSm<)EubcyI -> score: 245
mPSm<)EubcyI -> score: 245
mPSl<)EubcyI -> score: 244
mPSl<)EvbcyI -> score: 243
mPSl=)EvbcyI -> score: 242
mPSl=*EvbcyI -> score: 241
mPSl=*EvbcyH -> score: 242
mPSl=*EvbcyI -> score: 241
mPSl=*EvbcyI -> score: 241
mPSl=*EwbcyI -> score: 240
mPSm=*EwbcyI -> score: 241
mPSm=*ExbcyI -> score: 242
mPSm=*ExbcyI -> score: 242
mPSm=*ExbczI -> score: 243
mPSm=*ExbdzI -> score: 242
mPSm=*ExbczI -> score: 243
mPSm=*ExbczI -> score: 243
mPSm=*ExaczI -> score: 244
mPSm=*ExaczJ -> score: 243
mPSm>*ExaczJ -> score: 242
mPSm>*ExabzJ -> score: 243
mPSm=*ExabzJ -> score: 244
mPSm=*DxabzJ -> score: 243
mPSm=*DxabzI -> score: 244
mPSm<*DxabzI -> score: 245
mPSm<*DxabzI -> score: 245
mQSm<*DxabzI -> score: 244
mQSn<*DxabzI -> score: 245
m

oMVk6.At[bvN -> score: 249
oMVk7.At[bvN -> score: 248
nMVk7.At[bvN -> score: 247
nMVk7.As[bvN -> score: 248
nMVk7.As[bvN -> score: 248
nMUk7.As[bvN -> score: 249
nMUk7.As[bvN -> score: 249
nMUk7.As[avN -> score: 250
nMUk7.As[awN -> score: 251
nMUk7.As[awN -> score: 251
nMUk7.As[`wN -> score: 252
nMUk7.As[`wN -> score: 252
nMUk7-As[`wN -> score: 251
oMUk7-As[`wN -> score: 252
oMUk7-As[`wN -> score: 252
oMUk7-As[`wO -> score: 251
oMUj7-As[`wO -> score: 252
oMUj7-Bs[`wO -> score: 253
nMUj7-Bs[`wO -> score: 252
nMTj7-Bs[`wO -> score: 253
nMTj6-Bs[`wO -> score: 254
nMTj6-Bs[`wO -> score: 254
nMTj6-Cs[`wO -> score: 255
nNTj6-Cs[`wO -> score: 254
nNTj6,Cs[`wO -> score: 253
nNTj6,Cs[`wN -> score: 254
nNTj6,Ct[`wN -> score: 253
nNTj6,Ct[`wN -> score: 253
nNTj6,Ct[`wN -> score: 253
nNTj6,Ct\`wN -> score: 252
nNTj7,Ct\`wN -> score: 251
nNTj7,Ct\`wN -> score: 251
nNTj7,Ct\`wN -> score: 251
nNTj7,Ct\`wN -> score: 251
nNSj7,Ct\`wN -> score: 252
nMSj7,Ct\`wN -> score: 253
nMSj7,Cu\`wN -> score: 252
n

pQYi61Nzc`zG -> score: 266
pQYi61Ozc`zG -> score: 267
pQYi61Ozc`zG -> score: 267
pQYi61Nzc`zG -> score: 266
pQYi61Nzc`zG -> score: 266
pQXi61Nzc`zG -> score: 267
pQXi62Nzc`zG -> score: 268
pQXi62Nzc`zG -> score: 268
pRXi62Nzc`zG -> score: 267
pRXi62Ozc`zG -> score: 268
pQXi62Ozc`zG -> score: 269
pQXi62Ozc`zG -> score: 269
pQXi62Ozd`zG -> score: 268
pQXi72Ozd`zG -> score: 267
pQXi71Ozd`zG -> score: 266
pRXi71Ozd`zG -> score: 265
pSXi71Ozd`zG -> score: 264
pSXi71Ozd`zG -> score: 264
pSXi70Ozd`zG -> score: 263
pTXi70Ozd`zG -> score: 262
pTXi70Ozd`zG -> score: 262
pTXi70Ozd`zG -> score: 262
pTYi70Ozd`zG -> score: 261
pTYi71Ozd`zG -> score: 262
pTYi72Ozd`zG -> score: 263
pTYi72Ozd`zF -> score: 264
pTYi72Ozd_zF -> score: 265
pTYi72Ozd_zF -> score: 265
pTYi82Ozd_zF -> score: 264
pTYi81Ozd_zF -> score: 263
pTYi81Ozd^zF -> score: 264
pTYi81Ozd_zF -> score: 263
pTYi81Ozd_zE -> score: 264
pTYi81Ozd_zE -> score: 264
pTYi81Ozd_zE -> score: 264
pTYi81Ozd_zE -> score: 264
pTZi81Ozd_zE -> score: 263
p

yYVf..Wz^XxH -> score: 297
yYVf..Wz^XxH -> score: 297
yYVf..Wz^XxH -> score: 297
yYWf..Wz^XxH -> score: 296
yYWf..Wz^WxH -> score: 297
yYWf..Wz]WxH -> score: 298
yYWf..Wz]VxH -> score: 299
yYWf-.Wz]VxH -> score: 300
yYWf-.Wz]VwH -> score: 299
yYWf-.Wz]VwH -> score: 299
xYWf-.Wz]VwH -> score: 298
xYWf-.Wz]VwH -> score: 298
xYWf-.Wz]VwI -> score: 297
xYWe-.Wz]VwI -> score: 298
xYWe-.Wz]VwI -> score: 298
xYVe-.Wz]VwI -> score: 299
xZVe-.Wz]VwI -> score: 298
xZVe-.Wz]VwI -> score: 298
xZVe-.Wz]VwI -> score: 298
xZVe-.Wy]VwI -> score: 297
xZVe-.Wy]VvI -> score: 296
xZVe-.Wy]VvI -> score: 296
xZUe-.Wy]VvI -> score: 297
wZUe-.Wy]VvI -> score: 296
wZUe-.Wy]VvH -> score: 297
wZUe-.Wx]VvH -> score: 296
wZUe-.Wx]VvH -> score: 296
wZVe-.Wx]VvH -> score: 295
wZVe-.Wx]WvH -> score: 294
wZVe-.Wx]WvH -> score: 294
wZVe-.Wx]WvH -> score: 294
wZVe-.Wx]WuH -> score: 293
wZUe-.Wx]WuH -> score: 294
wZUe-.Wx]WuH -> score: 294
wZUe-.Wx]WuH -> score: 294
wZUe-.Wx]WvH -> score: 295
wZUd-.Wx]WvH -> score: 296
w

...

b^VW#9]HEa\^ -> score: 354
c^VW#9]HEa\^ -> score: 355
c^VW"9]HEa\^ -> score: 356
c^VX"9]HEa\^ -> score: 355
c^VX"9]HEa]^ -> score: 354
c^VX"9]HEa]^ -> score: 354
c^VX"9]HFa]^ -> score: 353
c^VX"9]HFa]^ -> score: 353
c^VY"9]HFa]^ -> score: 352
c^VY"9]HFa]^ -> score: 352
c^VY"9]HFa]^ -> score: 352
c_VY"9]HFa]^ -> score: 351
c_VY"9]HFa]^ -> score: 351
c_VX"9]HFa]^ -> score: 352
c_VY"9]HFa]^ -> score: 351
c_VY"9]HFa]^ -> score: 351
d_VY"9]HFa]^ -> score: 352
d_VY"9]HFa]] -> score: 353
d`VY"9]HFa]] -> score: 352
d`VY"9]HFa^] -> score: 351
d`VY"9]HFa^] -> score: 351
d`VY":]HFa^] -> score: 352
daVY":]HFa^] -> score: 351
daVY":]HFa^] -> score: 351
daVY":]GFa^] -> score: 352
daVY"9]GFa^] -> score: 351
daVY"9]HFa^] -> score: 350
daVY"9]HFa^] -> score: 350
dbVY"9]HFa^] -> score: 349
dbVY"9]HFa^] -> score: 349
dbVY"9]HFa^] -> score: 349
dbVX"9]HFa^] -> score: 350
dbVX"9]HFa^] -> score: 350
dbVX"9]IFa^] -> score: 349
dbVX"9]IFa_] -> score: 348
dbVX"9]IF`_] -> score: 349
dbWX"9]IF`_] -> score: 348
d

ci`V%<VLP[cd -> score: 315
ci`V%<VLP[dd -> score: 314
ci`V%<VLP[ed -> score: 313
ci`V%<VLP[ed -> score: 313
ciaV%<VLP[ed -> score: 312
ci`V%<VLP[ed -> score: 313
ci`V%<VLP[ed -> score: 313
ci`V%<VLP[ed -> score: 313
ci`V%<VLP[dd -> score: 314
ci`U%<VLP[dd -> score: 315
ci`U%<VLP[cd -> score: 316
ci`U%<VLQ[cd -> score: 315
ci`U%<VLQ[cd -> score: 315
ci`U%<VLQ[dd -> score: 314
ci`U%<VLR[dd -> score: 313
ci`U%<VLR[dd -> score: 313
ci`U%<VLR\dd -> score: 312
cj`U%<VLR\dd -> score: 313
cj`U%<WLR\dd -> score: 314
cj`U%<WLR\cd -> score: 315
cj`U%<WLS\cd -> score: 314
cj`U%<WLS\ce -> score: 315
cjaU%<WLS\ce -> score: 314
cjaU%<WMS\ce -> score: 313
cjaU%<WMS\ce -> score: 313
cjaU%<WLS\ce -> score: 314
cjaU%<VLS\ce -> score: 313
cjaU%<VKS\ce -> score: 314
cjaU%<UKS\ce -> score: 313
cjaU%<UKT\ce -> score: 312
cjaU%<UKT\ce -> score: 312
ciaU%<UKT\ce -> score: 311
ciaU%<UKS\ce -> score: 312
ciaU%<UKS[ce -> score: 313
ciaU%<UKS[cd -> score: 312
ciaU$<UKS[cd -> score: 313
ciaU$<UKS[dd -> score: 312
c

\kgX)4UJYilh -> score: 262
\kgX)5UJYilh -> score: 263
\kgX)5UJYilh -> score: 263
\kgX)5UJYilh -> score: 263
\kgX(5UJYilh -> score: 264
\kgX(5UJYilh -> score: 264
\kgX(5UJYilh -> score: 264
\khX(5UJYilh -> score: 263
\khX(5UJYilh -> score: 263
\khX(5UJYilh -> score: 263
\khX(4UJYilh -> score: 262
\khX(4UJYilh -> score: 262
\khX(4UJYilh -> score: 262
\khX(4UKYilh -> score: 261
\khX(4UKYjlh -> score: 260
\khX(4UKYjlh -> score: 260
\kgX(4UKYjlh -> score: 261
\kgX(4UKXjlh -> score: 262
\kgX(4UKXjlh -> score: 262
\kgX(4UKXjlh -> score: 262
\kgY(4UKXjlh -> score: 261
\kgY(4ULXjlh -> score: 260
\kgY)4ULXjlh -> score: 259
\kgY)4ULXjlh -> score: 259
\kgY)4TLXjlh -> score: 258
\kgY)4TLXjlh -> score: 258
\kgY)4TLXklh -> score: 257
\kgY)4TLXklh -> score: 257
\kgY)4TMXklh -> score: 256
\kgY)4SMXklh -> score: 255
\kgY)4SMXklh -> score: 255
\kfY)4SMXklh -> score: 256
\kfY)4SMXkmh -> score: 257
\kfY)5SMXkmh -> score: 258
\kfY)5SMXkmh -> score: 258
\kfY)5SMXkmh -> score: 258
\kfY)5SMXkmh -> score: 258
\

Ifl_2,M[cnld -> score: 165
Ifl_2,M[cnld -> score: 165
Ifl_2,M\cnld -> score: 164
Ifl_2,M\cnld -> score: 164
Igl_2,M\cnld -> score: 165
Igl_2,M\cnld -> score: 165
Igl_2,M\cnld -> score: 165
Igl_2,M\cnld -> score: 165
Igl_2,M\cnld -> score: 165
Igl_2,M\cnld -> score: 165
Igl_2,M\cnld -> score: 165
Igl_2,M\cnld -> score: 165
Igl`2,M\cnld -> score: 164
Igl`2,M\cnld -> score: 164
Igla2,M\cnld -> score: 163
Igla2,M\cnld -> score: 163
Igla2,M\cnlc -> score: 164
Igla2,M\cnlc -> score: 164
Igla2,M\cnlc -> score: 164
Igla2,M\cnlc -> score: 164
Igla1,M\cnlc -> score: 165
Igla1,M\dnlc -> score: 164
Ihla1,M\dnlc -> score: 165
Ihla1,M\dnlc -> score: 165
Ihla1,M\dnlc -> score: 165
Ihla1,L\dnlc -> score: 164
Jhla1,L\dnlc -> score: 165
Jhla1,L\dnlc -> score: 165
Jhla1,L\dmlc -> score: 166
Jhla1,L\emlc -> score: 165
Jhla1,L\emlc -> score: 165
Jgla1,L\emlc -> score: 164
Jgla1,L\emlc -> score: 164
Jgla1,L[emlc -> score: 165
Jgla1,L[emlc -> score: 165
Jglb1,L[emlc -> score: 164
Jglb1,L[emlc -> score: 164
J

HellD,5world -> score: 64
HellE,5world -> score: 63
HellE,5world -> score: 63
HellE,5world -> score: 63
HellE,5world -> score: 63
HellE,5world -> score: 63
HellE,5world -> score: 63
HellE,5world -> score: 63
HellE,5world -> score: 63
HellE,5world -> score: 63
HellE,5world -> score: 63
HellE,5world -> score: 63
HellE,5world -> score: 63
HellE,5world -> score: 63
HellE,5world -> score: 63
HellE,5world -> score: 63
HellE,5world -> score: 63
HellF,5world -> score: 62
HellF,5world -> score: 62
HellF,4world -> score: 61
HellF,4world -> score: 61
HellF,4world -> score: 61
HellF,4world -> score: 61
HellF,4world -> score: 61
HellF,4world -> score: 61
HellF,4world -> score: 61
HellG,4world -> score: 60
HellG,4world -> score: 60
HellG,4world -> score: 60
HellG,4world -> score: 60
HellG,4world -> score: 60
HellG,4world -> score: 60
HellG,4world -> score: 60
HellG,4world -> score: 60
HellG,4world -> score: 60
HellG,4world -> score: 60
HellG,3world -> score: 59
HellG,3world -> score: 59
HellG,3world

In [13]:
print()
print("*************Simulated annealing***************")
print("steps:{}, best solution:'{}', best score:{}".format(iterations,best_sol,best_score))


*************Simulated annealing***************
steps:15102, best solution:'Hello, world', best score:0


## 4. Genetic algorithm

This optimization technique is inspired by the theory of evolution. The algorithm starts with a population of random individuals, and selects the ones with the best fitness score (elite). It continues to the next generation with this group. In order to enrich the genetic pool in the current generation, the algorithm adds random mutations and crossover to the elite group. After predefined number of generations, the algorithm returns the top-fit individual. 

### 4.1. Mutation and crossover

In [14]:
# Mutation Operation
def string_mutation(individual):
    i = random.randint(0, len(individual) - 1)

    # mutation changes character at random position to any valid character   
    rchar = chr(random.randint(32, 122))

    individual = individual[0:i] + rchar + individual[(i + 1):]
    return individual


# Mate operation (crossover)
def string_crossover (s1, s2):
    # find the point of crossover
    i = random.randint (0, len(s1)-1)
    return s1[:i]+s2[i:]

### 4.2. Algorithm

Initial population - a list of random strings

In [16]:
def genetic_optimize(population, fitness_function,
                     mutation_function, mate_function,
                     mutation_probability, elite_ratio,
                     maxiterations):

    # How many winners to consider from each generation?
    original_population_size = len(population)
    top_elite = int(elite_ratio * original_population_size)

    # Main loop
    iterations = 0
    for i in range(maxiterations):
        iterations += 1
        individual_scores = [(fitness_function(v), v) for v in population]
        individual_scores.sort()
        ranked_individuals = [v for (s, v) in individual_scores]

        # Start with the pure winners
        population = ranked_individuals[0:top_elite]

        # Add mutated and bred forms of the winners
        while len(population) < original_population_size:
            if random.random() < mutation_probability:
                # Mutation
                c = random.randint(0, top_elite)
                population.append(mutation_function(ranked_individuals[c]))
            else:
                # Crossover
                c1 = random.randint(0, top_elite)
                c2 = random.randint(0, top_elite)
                population.append(mate_function(ranked_individuals[c1], ranked_individuals[c2]))

        # Print current best score
        if print_steps:
            print(individual_scores[0][1]," -> score:", individual_scores[0][0])

        if individual_scores[0][0] == 0:
            return (individual_scores[0][0],individual_scores[0][1], iterations)

    # returns the best solution
    return (individual_scores[0][0], individual_scores[0][1], iterations)



string_population = get_rand_population(2048, len(s), alphabet)
best_score, best_sol, iterations  = genetic_optimize(string_population, string_fitness,
                                string_mutation, string_crossover,
                                mutation_probability=0.25, elite_ratio=0.1,
                                maxiterations=100)
print()
print("*****************GENETIC ALGORITHM ***************")
print("generations:{}, best solution:'{}', best score:{}".format(iterations,best_sol,best_score))


GRitd4$Wtsll  -> score: 100
GRitd4$Wtsll  -> score: 100
GRitd4$qtslT  -> score: 82
Reilz1-xisll  -> score: 58
Hh\lp(#uashd  -> score: 48
Hhilt0#utsl`  -> score: 30
Hhilp(#ulrhd  -> score: 23
Heilo)$uoshd  -> score: 17
Hhilp. xoshd  -> score: 15
Heilp. upsld  -> score: 10
Heilo) xosld  -> score: 8
Heilo. xosld  -> score: 7
Heilo+ xosld  -> score: 6
Hello- xosld  -> score: 3
Hello- xosld  -> score: 3
Hello, xosld  -> score: 2
Hello, xosld  -> score: 2
Hello+ wosld  -> score: 2
Hello, wosld  -> score: 1
Hello, wosld  -> score: 1
Hello, world  -> score: 0

*****************GENETIC ALGORITHM ***************
generations:21, best solution:'Hello, world', best score:0


### This is the end of the ''Hello, world'' demo.
Copyright &copy; 2022 Marina Barsky